# Práctica 3: Resolución Numérica  de PVIs  usando Python

Autor: Pedro González Rodelas

Fecha de la primera versión: 01/05/2020

Sucesivas revisiones: 02-08/05/2020

Fecha de la última revisión: 11/05/2020

En esta práctica se programan en lenguaje Python diferentes métodos numéricos de resolución aproximada de Problemas de Valores Iniciales asociados a Ecuaciones Diferenciales Ordinarias (EDOs) y sistemas de primer orden, provenientes muchos de ellos de problemas y aplicaciones interesantes de Física e Ingeniería.

Algunos de los ejemplos y algoritmos usados en este notebook han sido extraidos, adaptados o traducidos por Pedro González Rodelas de alguno de los siguientes libros y/o apuntes:


- "Análisis Numérico con Mathematica" de V. Ramírez, D. Barrera, M. Pasadas y P. González. Edit. Ariel Ciencia (2001) con ISBN: 84-344-8036-0. Dep. Legal: B.31.425-2001.

- Apuntes de la asignatura de Métodos Numéricos II del Grado de Matemáticas y Dobles Grados de Matemáticas e Informática o Física de la UGR, de los profesores José Martínez Aroza y Victoriano Ramírez González.

- Notebook perteneciente al Capítulo 9 del libro [Numerical Python - A Practical Techniques Approach for Industry](http://www.apress.com/9781484205549) (ISBN 978-1-484205-54-9) de Robert Johansson.

- Capítulo 7 del libro [Numerical Methods in Engineering with Python 3](https://www.cambridge.org/core/books/numerical-methods-in-engineering-with-python-3/95151C37C2F427F30DC90FA619FE79F9) de Jaan Kiusalaas.

In [ ]:
# Cargando el fichero version_information.py
# alojado el directorio de trabajo actual
# podremos comprobar las versiones concretas
# tanto del intérprete de Python, como de los
# módulos que tenemos instalados en nuestro ordenador.

%reload_ext version_information

%version_information numpy, matplotlib, sympy, scipy

## Resolviendo EDOs simbolicamente con SymPy

Como es bien sabido, una ecuación diferencial ordinaria (EDO), no es más que una relación existente entre algunas derivadas de función incógnita $y(t)$, suponiendo que resulta ser suficientemente derivable, a partir de la función $f\equiv f(t,z)$ que determina la EDO que satisface. 

En el caso de una EDO de primer orden esta relación se podría escribir de la siguiente manera genérica: $$y'(t)=f(t,y(t)),\; t\in[a,b]$$
con $f:[a,b]\times\mathbb{R}\to \mathbb{R}$ también suficientemente regular, o al menos continua en $(t,z)$ y Lipschitziana en $z$, para asegurar existencia y unicidad de solución del correspondiente Problema de Valores Iniciales (PVI), para cierta condición inicial $y(a) = y_a\equiv\mu\in\mathbb{R}$ determinada de antemano:

$$\begin{array}{c l} y'(t)=f(t,y(t)),& t\in[a,b]\\ y(a) = y_a\equiv\mu  &  \end{array}$$

aunque también suele ser habitual el empleo de la siguiente notación

$$\begin{array}{c l} x'(t)=f(t,x(t)),& t\in[a,b]\\ x(a) = x_0\equiv\mu  &  \end{array}$$

o usar $x$ como variable independiente e $y$ como variable dependiente, etc. 

Así pues, con el objeto de normalizar la notación y que valga para cualquier caso, hemos optado por usar una notación genérica que valga para cualquier EDO, independientemente de la notación empleada para la misma.

In [ ]:
import sympy as sp
# sp.init_printing()

In [ ]:
t, y0, z = sp.symbols('t, y_0,  z')
def f(t,z):
    return z

In [ ]:
a = 0; b = 1;   # extremos inferior y superior del intervalo 
ya = 1          # condición inicial del PVI

In [ ]:
# Vamos a empezar definiendo la Ec. Dif. Ordinaria (edo) a resolver
y = sp.Function('y')
edo = y(t).diff(t)-f(t,y(t))
sp.Eq(edo,0)

In [ ]:
# y la resolvemos simbólicamente mediante la orden dsolve de SymPy
edo_sol = sp.dsolve(edo)
edo_sol

In [ ]:
# De esta solución simbólica obtenida también podremos extraer, tanto
edo_sol.lhs  #  el término de la izquierda (mediante .lhs)

In [ ]:
edo_sol.rhs  #  como el término de la derecha (mediante .rhs)

In [ ]:
# Veámos ahora cómo imponer las condiciones iniciales a través
ics = {y(0): y0}  #  de una entrada de diccionario
ics

In [ ]:
# De esta manera haríamos la sustituciones correspondientes
edo_sol.rhs.subs(t, 0),edo_sol.lhs.subs(t, 0).subs(ics)

In [ ]:
CI_eq = sp.Eq(edo_sol.lhs.subs(t, 0).subs(ics),edo_sol.rhs.subs(t, 0))
CI_eq   # esta sería finalmente la ecuación asociada 
# a la imposición de la condición inicial del PVI

In [ ]:
CI_sol = sp.solve(CI_eq) # que resolvemos mediante la orden solve
CI_sol[0]                # y extraemos la única solución de ésta

In [ ]:
edo_sol   # Usaremos finalmente la solución analítica general

In [ ]:
# así como la obtención de las constantes de integración
edo_sol.subs(CI_sol[0]) # en función de las condiciones iniciales

In [ ]:
# para obtener la única solución exacta de nuestro PVI 
sol_exacta = edo_sol.rhs.subs(CI_sol[0]).subs({y0:ya})
sol_exacta

### Función genérica para aplicar las condiciones  iniciales de un PVI

Podríamos implementar ahora de una manera sistemática esta técnica de obtención simbólica de la solución analítica de un PVI, aprovechando al máximo la potencia y flexibilidad que nos aporta el lenguaje Python, así como su módulo de cálculo simbólico por excelencia SymPy.

In [ ]:
def apply_ics(sol, ics, x, known_params):
    """
    Applicamos las condiciones iniciales (CIs), dadas como un diccionario
    de la forma ics = {y(0): y0: y(x).diff(x).subs(x, 0): yp0, ...}
    a la solución de la EDO con en la variable independiente x.
    Las constantes de integración indeterminadas C1, C2, ... serán extraidas
    a partir de los símbolos libres de la solución de la EDO, excluyendo 
    aquellos de la lista de parametros conocidos 'known_params'.
    """
    free_params = sol.free_symbols - set(known_params)
    eqs = [(sol.lhs.diff(x, n) - sol.rhs.diff(x, n)).subs(x, 0).subs(ics)
           for n in range(len(ics))]
    sol_params = sympy.solve(eqs, free_params)
    return sol.subs(sol_params)

## Importación del módulo NumPy

La importación de este módulo será completamente imprescindible en casi todas las implementaciones desarrolladas a lo largo de esta práctica, al ser el módulo de Python fundamental para cualquier empleo eficiente de métodos numéricos en este lenguaje de programación; pero también resultará útil para la obtención de los arrays de abcisas y ordenadas de cualquier gráfico obtenido al estilo de MATLAB u Octave.

In [ ]:
# Como es habitual realizaremos la importación con el pseudónimo np
import numpy as np  

## Representación gráfica de las soluciones

In [ ]:
# Configuramos e importamos el submódulo Pyplot de Matplotlib
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['text.usetex'] = True

In [ ]:
sol_exacta # recordamos cuál era la solución exacta de nuestro PVI

In [ ]:
# para poder evaluarla en el valor de la variable independiente 
sol_exacta.subs({t:1}).evalf()  # t que queramos

In [ ]:
print(sol_exacta)  # esta es la expresión que podríamos pegar donde queramos

In [ ]:
a,b    # el intervalo de trabajo

In [ ]:
m = sol_exacta.subs({t:a})  # valores de referencia a la izqda.
M = sol_exacta.subs({t:b})  # y derecha del intervalo

In [ ]:
# para representar gráficamente expresiones simbólicas
sp.plot?

In [ ]:
graf_sympy = sp.plot(sol_exacta, xlim =(a,b), ylim = (0,M));

In [ ]:
# en caso de que queramos extender este gráfico simbólico con otro 
graf_sympy.append(sp.plot(t,(t,a,b),show=False)[0])
graf_sympy.show()

Aunque siempre tendremos la posibilidad de hacerlo de la manera tradicional habitual en MATLAB u Octave, pero esto requeriría reconvertir previamente esta solución analítica, dada simbólicamente, en una función numérica, evaluable de manera vectorial en arrays de valores, como por ejemplo las abcisas de la variable independiente en los que queramos obtener sus correspondientes ordenadas.

In [ ]:
# Esta sería la herramienta adecuada para realizar dicha conversión
y_x = sp.lambdify(t, sol_exacta, 'numpy')

In [ ]:
xx = np.linspace(a, b, 100) # así obtendríamos el array de abcisas

In [ ]:
y_x(xx)   # y así el correspondiente array de ordenadas

In [ ]:
# y ya estaría todo preparado para poder realizar la correspondiente
fig, ax = plt.subplots(figsize=(6, 6))  # figura de Matplotlib
xx = np.linspace(a, b, 100)
ax.plot(xx, y_x(xx), label="sol. exacta")
ax.set_title(r"$%s$" % sp.latex(edo_sol), fontsize=18)
ax.set_xlabel(r"$t$", fontsize=18)
ax.set_ylabel(r"$y$", fontsize=18)
ax.legend()

fig.tight_layout()

In [ ]:
graf_exacta = fig  # de esta manera podremos volver a generar esta figura
# cuando lo necesitemos, por ejemplo para compararla con las aproximaciones

## Módulo de impresión de resultados numéricos

Como posteriormente vamos a necesitar imprimir y/o mostrar los resultados obtenidos mediante la aplicación de diferentes métodos numéricos, hemos optado por definir una herramienta común que grabaremos en un fichero .py, que usaremos a modo de módulo particular, que nos permitirá usarlo repetidamente de una manera sistemática, y no tener que volver a repetir esta porción de código cada vez que lo necesitemos a lo largo de la práctica.

In [ ]:
%pwd  # para saber en qué directorio estamos trabajando

Vemos que con estos denominados comandos ``mágicos`` (precedidos de `%`) podemos interactuar con el sistema operativo en el que estemos ejecutando este notebook, para conocer en qué directorio nos encontramos, listar, borrar o crear nuevos archivos, cambiar de directorio, etc., como veremos a continuación.  

In [ ]:
%%file printSoln.py
## module printSoln
''' imprimeSol(X,Y,freq).
    Imprime los arrays X e Y devueltos por los códigos
    de resolución de una ecuación diferencial concreta
    usando una frecuencia de salida especificada, 'freq'.
        freq = n imprime cada n pasos.
        freq = 0 imprime sólo los valores inicial y final.
'''
def printSoln(X,Y,freq):

    def printHead(n):
        print("\n        x  ",end=" ")
        for i in range (n):
            print("      y[",i,"] ",end=" ")
        print()

    def printLine(x,y,n):
        print("{:13.4e}".format(x),end=" ")
        for i in range (n):
            print("{:13.4e}".format(y[i]),end=" ")
        print()
    
    m = len(Y)
    try: n = len(Y[0])
    except TypeError: n = 1
    if freq == 0: freq = m
    printHead(n)
    for i in range(0,m,freq):
        printLine(X[i],Y[i],n)
    if i != m - 1: printLine(X[m - 1],Y[m - 1],n)


In [ ]:
# comprobamos a continuación que efectivamente se ha creado dicho archivo
%ls *.py



# Métodos de Euler

In [ ]:
%%file   eulerexplicito.py
## módulo euler
''' método de Euler para resolver el PVI
    X,Y     = integrate(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
'''
import numpy as np
def integrate(F,x0,y0,xfinal,N):    
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]; h = (xfinal-x0)/N
    for n in range(N):
        Y.append(Y[n] + h*F(X[n],Y[n]))
    return np.array(X),np.array(Y)

In [ ]:
# para usar el código anterior guardado en el fichero eulerexplicito.py
import eulerexplicito

In [ ]:
# Si posteriormente tuviéramos que volver a importarlo, con Pyhton 3
# import importlib    # necesitaríamosimportar previamente este módulo
# importlib.reload(module)
# importlib.reload(eulerimplicito) # y ya podríamos cargarlo de nuevo cuando queramos

In [ ]:
%ls eulerexplicito.py

In [ ]:
help(eulerexplicito)

In [ ]:
# Apliquemos pues ahora este algoritmo del método de Euler explícito
N=10;  # con un valor concreto de N y la función f que define nuestra EDO
xx, yEulerexpl = eulerexplicito.integrate(f,a,ya,b,N)
# y asignémosle los arrays que devuelve a las abcisas y ordenadas

In [ ]:
print(yEulerexpl)  #  aprox. obenidas en los nodos considerados

In [ ]:
# que justo a continuación podremos dibujar y comparar con la sol. exacta
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(xx,yEulerexpl, 'bo',label = "aprox. mét. Euler explícito")
ax.plot(xx, y_x(xx), label="sol. exacta")
ax.set_title(r"Sol. exacta:  $y(t)=%s$" % sp.latex(sol_exacta), fontsize=18)
ax.set_xlabel(r"$t$", fontsize=18)
ax.set_ylabel(r"$y$", fontsize=18)
ax.legend()
fig.tight_layout()
graf_Eulerexpl = fig

In [ ]:
%%file eulerimplicito.py
## módulo eulerimplicito
''' método de Euler implicito para resolver el PVI
    X,Y     = integrate(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
'''
import numpy as np
import sympy as sp
def integrate(F,x0,y0,xfinal,N):    
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]; h = (xfinal-x0)/N
    y1 = sp.Symbol('y1') # usaremos esta variable como incógnita
    for n in range(N):   # de la ecuación a resolver en cada iteración
        Y.append(sp.solve(Y[n]+h*F(X[n+1],y1)-y1,y1)[0])
    return np.array(X),np.array(Y)

In [ ]:
import eulerimplicito  # así importaríamos el contenido guardado en euler.py

In [ ]:
%%file eulerimplicitoaprox.py
## módulo eulerimplicitoaprox
''' método de Euler implicito aprox para resolver el PVI
    X,Y     = integrate(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
'''
import numpy as np
import sympy as sp

def integrate(F,x0,y0,xfinal,N):    
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]; h = (xfinal-x0)/N
    
    for n in range(N):   
        y1 = Y[n] + h*F(X[n],Y[n])    #  valor inicial para una
        Y.append(Y[n]+h*F(X[n+1],y1)) #  iteración de punto fijo
    return np.array(X),np.array(Y)

In [ ]:
# %ls euler*.py

In [ ]:
import eulerimplicitoaprox  # así importaríamos el contenido guardado en euler.py

In [ ]:
# Si posteriormente tuviéramos que volver a importarlo, con Pyhton 3
# import importlib    # necesitaríamosimportar previamente este módulo
# importlib.reload(module)
# importlib.reload(eulerimplicito) # y ya podríamos cargarlo de nuevo cuando queramos

In [ ]:
# Apliquemos pues ahora alguna de estas versiones del algoritmo 
N=10;  # del método de Euler implícito, con un valor concreto de n
xx, yEulerimpl = eulerimplicito.integrate(f,a,ya,b,N)
xx, yEulerimplaprox = eulerimplicitoaprox.integrate(f,a,ya,b,N)
# y asignémosle los arrays que devuelve a las abcisas y ordenadas

In [ ]:
print(yEulerimpl)  #  aprox. obenidas en los nodos considerados

In [ ]:
print(yEulerimplaprox)  #  aprox. obenidas en los nodos considerados

In [ ]:
# que justo a continuación podremos dibujar y comparar con la sol. exacta
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(xx,yEulerimpl, 'bo',label = "aprox. mét. Euler implícito")
ax.plot(xx,yEulerimplaprox, 'r-.D',label = "aprox. mét. Euler implícito aprox.")
ax.plot(xx, y_x(xx), label="sol. exacta")
ax.set_title(r"Sol. exacta:  $y(t)=%s$" % sp.latex(sol_exacta), fontsize=18)
ax.set_xlabel(r"$t$", fontsize=18)
ax.set_ylabel(r"$y$", fontsize=18)
ax.legend()
fig.tight_layout()
graf_Eulerimpl = fig

In [ ]:
# Al haberle puesto nombre a esta gráfica podremos volver a reproducirla
# graf_Eulerimpl     # sin más que acordarnos de su nombre

## Comparativa entre los métodos de Euler explícito e implícito

In [ ]:
valores_exactos = np.array([sol_exacta.subs({t:xx[n]}) for n in range(len(xx))])
print(valores_exactos)

In [ ]:
abs(valores_exactos - yEulerexpl).max() # este sería el valor absoluto 
# máximo de las diferencias entre los valores exactos y los aproximados 
# mediante el uso del método de Euler explícito

In [ ]:
abs(valores_exactos - yEulerimpl).max() # este sería el valor absoluto 
# máximo de las diferencias entre los valores exactos y los aproximados 
# mediante el uso del método de Euler implícito

In [ ]:
abs(valores_exactos - yEulerimplaprox).max() # este sería el valor absoluto 
# máximo de las diferencias entre los valores exactos y los aproximados 
# mediante el uso del método de Euler implícito

**Ejercicios:**
1. Repita las aproximaciones anteriores con diferentes valores de $N$ (y por tanto de $h$) y compruebe el efecto en cuanto a mayor o menor precisión, estabilidad y coste computacional 
1. A partir de la implementación del algoritmo del método de Euler explícito, realice las modificaciones oportunas para obtener también las implementaciones correspondientes a los métodos de Euler mejorado (o del punto medio), así como del de Euler modificado (o de Heun).

**Nota importante:** en caso de necesitar una sóla iteración (o el valor $y_1$) del método que fuera entonces convendría adaptar los algoritmos anteriores, así como todos los que vienen a continuación, para que por ejemplo introducidos `(f,a,ya,b,N)` tan sólo calculen y devuelvan `Y[1]` o directamente `y1`, con el corespondiente ahorro de tiempo de cómputo. Ver ejemplo a continuación.

In [ ]:
%%file eulerexplrapido.py
## módulo eulerexplrapido
''' método de Euler para resolver el PVI
    X,Y     = integrate(F,x0,y0,x1,h).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1]}.
    x0,y0   = condiciones iniciales 
    x1      = valor final de la variable x0 + h en este caso
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario
'''
import numpy as np
def integrate(F,x0,y0,h):    
    return y0 + h*F(x0,y0)

In [ ]:
import eulerexplrapido

Habría una posibilidad de intentar estimar el tiempo de CPU empleado 

In [ ]:
import timeit

In [ ]:
N = 10; h = (b-a)/N
%time
eulerexplrapido.integrate(f,a,ya,h)

In [ ]:
%%timeit
eulerexplrapido.integrate(f,a,ya,h)

In [ ]:
%time
eulerexplicito.integrate(f,a,ya,b,N)

In [ ]:
%%timeit
eulerexplicito.integrate(f,a,ya,b,N)

Al usar `%time` el intérprete de Python nos intenta devolver el tiempo empleado en la ejecución de dicho comando (o linea de comandos), pero esto suele depender muchas veces de la carga puntual de la CPU en ese justo momento de la ejecución; por ese motivo se suele preferir el uso del comando mágico `%%timeit` que lo que hace más bien es realizar una tanda de ejecuciones repetitivas de dicha linea o bloque de comandos, tantas veces como sea necesario para obtener un valor estimado promedio, que es el que nos devuelve seguidamente. Habrá que tener en cuenta en todo caso, tanto las unidades de medida del tiempo que nos devuelva (ya sea en  nano- ($10^{-9}$ s), micro- ($10^{-6}$ s), mili-segundos ($10^{-3}$ s), etc), como el número de veces que se ha llevado a cabo la tanda de repeticiones, que puede oscilar entre unos cientos o miles, hasta a veces un millón, para poder comparar finalmente de una manera realista la rapidez de ejecución de cierto trozo de código.

# Métodos de Taylor

Como es bien sabido, los métodos de Taylor se basan en un adecuado desarrollo en serie de Taylor de la propia función incógnita $y(t)$, suponiendo que resulta ser suficientemente derivable, y que sus derivadas sucesivas se pueden obtener a partir de la función $f$ que determina la EDO que satisface: $$y'(t)=f(t,y(t)),\; t\in[a,b]$$

Así pues, si suponemos que $y\in\mathcal{C}^p$ y que podemos realizar un desarrollo en serie como el que sigue:
$$y(t+h)\approx y(t)+h\, y'(t)+\frac{h^2}{2}\, y''(t)+\ldots+\frac{h^p}{p!}\, y^{p)}(t)\\ \equiv y(t)+h\, F_1(t,y(t))+\frac{h^2}{2}\, F_2(t,y(t))+\ldots+\frac{h^p}{p!}\, F_p(t,y(t))$$

pues justamente el conocido método de Euler explícito, visto anteriormente, correspondería con el desarrollo de Taylor correspondiente al caso $p=1$, definiendo por ejemplo $F_1(t,z)\equiv f(t,z)$. 

Así pues vamos por ejemplo a centrarnos en desarrollar por ejemplo, el siguiente caso $p=2$, y los demás casos $p>2$ se harían de la misma manera, sólo que conllevarían la realización de más cálculos del mismo tipo.

**Caso $p=2$:**
Aquí de lo que se trataría es de aprovechar la expresión de la EDO que relaciona la derivada primera de $y(t)$ con $f(t,y(t))$, y realizar la derivada segunda de acuerdo con el siguiente desarrollo (asumiendo que $f\equiv f(t,z)$):

$$y''(t)=\frac{d}{dt}y'(t)=\frac{d}{dt}(f(t,y(t)))=\frac{df}{dt}(t,y(t))+\frac{df}{dz}(t,y(t))\,y'(t) \\= \frac{df}{dt}(t,y(t))+\frac{df}{dz}(t,y(t))\,f(t,y(t)) \equiv F_2(t,y(t))$$

de manera que finalmente nuestro métodos de Taylor de orden $p=2$ quedaría como sigue:

$$y(t+h)\approx y(t)+h\,F_1(t,y(t)) +\frac{h^2}{2}\, F_2(t,y(t))$$

con $F_1(t,z)\equiv f(t,z)$  y $F_2(t,z)\equiv \frac{df}{dt}(t,z)+\frac{df}{dz}(t,z)\,f(t,z)$.

Vamos pues a intentar implementar este método de Taylor de orden $p=2$ aprovechando las capacidadesd de cálculo simbólico que también nos proporciona el módulo de cálculo simbólico de Python, SymPy.

In [ ]:
f(t,z), sp.diff(f(t,z),t), sp.diff(f(t,z),z)

In [ ]:
def F1(t,z):
    return f(t,z)

def F2(t,z):
    return sp.diff(f(t,z),t) + sp.diff(f(t,z),z)*F1(t,z)

In [ ]:
F1(t,z), F2(t,z)

In [ ]:
%%file Taylor2.py
# -*- coding: UTF-8 -*-
## módulo Taylor2
''' método del método de Taylor de orden 2 para resolver el PVI
    X,Y     = integrate(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
'''
def integrate(F,x0,y0,xfinal,N):  
    import numpy as np
    import sympy as sp
    
    t, z = sp.symbols('t, z')
    
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]; h = (xfinal-x0)/N
    
    def F1(t,z):
        return F(t,z)

    def F2(t,z):
        return sp.diff(F(t,z),t) + sp.diff(F(t,z),z)*F1(t,z)
    
    for n in range(N):
        Y.append(Y[n]+h*F1(X[n],Y[n])+h**2/2*F2(t,z).subs({t:X[n],z:Y[n]}))
    return np.array(X),np.array(Y)

In [ ]:
# %ls *.py

In [ ]:
# usar 'cat' o 'more' en Linux o Mac OS 
# !type Taylor2.py

In [ ]:
f(t,z)

In [ ]:
import Taylor2

In [ ]:
# Si posteriormente tuviéramos que volver a importarlo, con Pyhton 3
# import importlib    # necesitaríamosimportar previamente este módulo
# importlib.reload(Taylor2) #y ya podríamos cargarlo de nuevo cuando queramos
# importlib.reload(eulerimplicito) # 

In [ ]:
f(t,z), sp.diff(f(t,z),t), sp.diff(f(t,z),z)

In [ ]:
# Apliquemos pues ahora este algoritmo del método de Taylor de orden 2
N=10;  # con un valor concreto de N
xx, yTaylor2 = Taylor2.integrate(f,a,ya,b,N)
# y asignémosle los arrays que devuelve a las abcisas y ordenadas

In [ ]:
print(yTaylor2)

In [ ]:
# que justo a continuación podremos dibujar y comparar con la sol. exacta
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(xx,yTaylor2, 'bo',label = "aprox. mét. Taylor de orden 2")
ax.plot(xx, y_x(xx), label="sol. exacta")
ax.set_title(r"Sol. exacta:  $y(t)=%s$" % sp.latex(sol_exacta), fontsize=18)
ax.set_xlabel(r"$t$", fontsize=18)
ax.set_ylabel(r"$y$", fontsize=18)
ax.legend()
fig.tight_layout()
graf_ptomedio = fig

**Ejercicios:**
1. Realice varios experimentos numéricos, con diferentes valores de $N$, y compare y ratifique los órdenes de convergencia de los diferentes métodos vistos hasta el momento.
 
1. A partir de la implementación del algoritmo del método de Taylor de orden $p=2$, realice las modificaciones oportunas para obtener también una posible implementación correspondiente al método de Taylor de orden $p=3$.

# Métodos de Runge-Kutta

## Runge-Kutta de orden 4

In [ ]:
%%file RK4.py
## módulo RK4
''' Método de Runge-Kutta de cuarto orden para resolver
    el problema de valores iniciales {y}' = {F(x,{y})}, 
    donde    {y} = {y[0],y[1],...y[N-1]}.
    X,Y    = integrate(F,x0,y0,xfinal,h).
    x0,y0  = condiciones iniciales
    xfinal = valor final de x
    N      = número de subintervalos de la partición
    h      = incremento de x usado en la integración
    F      = función suplida por el usuario que devuelve el
            array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
'''
import numpy as np
def integrate(F,x0,y0,xfinal,N):
    
    def RK4(F,xn,yn,h):
        K1 = F(xn,yn)
        K2 = F(xn + h/2, yn + K1*h/2)
        K3 = F(xn + h/2, yn + K2*h/2)
        K4 = F(xn + h  , yn + K3*h)
        return (K1 + 2*K2 + 2*K3 + K4)/6    
    
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]; h = (xfinal-x0)/N
    for n in range(N):
        Y.append( Y[n] + h*RK4(F,X[n],Y[n],h) )
    return np.array(X),np.array(Y)

In [ ]:
import RK4
# importlib.reload(RK4)

In [ ]:
# Apliquemos pues ahora este algoritmo del método de Euler
N=10;  # con un valor concreto de n
xx, yRK4 = RK4.integrate(f,a,ya,b,N)
# y asignémosle los arrays que devuelve a las abcisas y ordenadas

In [ ]:
# que justo a continuación podremos dibujar y comparar con la sol. exacta
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(xx,yRK4, 'bo',label = "aprox. mét. RK4")
ax.plot(xx, y_x(xx), label="sol. exacta")
ax.set_title(r"Sol. exacta:  $y(t)=%s$" % sp.latex(sol_exacta), fontsize=18)
ax.set_xlabel(r"$t$", fontsize=18)
ax.set_ylabel(r"$y$", fontsize=18)
ax.legend()
fig.tight_layout()
graf_RK4 = fig

**Ejercicios:**
1. Repita las aproximaciones anteriores con diferentes valores de $N$ (y por tanto de $h$) y compruebe el efecto en cuanto a mayor o menor precisión, estabilidad y coste computacional 
1. A partir de la implementación del algoritmo del método de Runge-Kutta de 4 evaluaciones, realice las modificaciones oportunas para intentar obtener también las implementaciones correspondientes a otros muchos métodos de tipo Runge-Kutta, tanto explícitos como implícitos, a partir del correspondiente arreglo de Butcher.

# Métodos Multipaso Lineales (MML)

Recordamos que un método multipaso para el PVI considerado en esta práctica, tendría la siguiente forma:
$$
y_0, y_1, \ldots, y_{k-1} \text{ valores iniciales de partida}\\
y_{n+k} = \sum_{j=0}^{k-1} \alpha_j \, y_{n+j} + h\, \Phi(y_{n+k}, y_{n+k-1}, \ldots, y_{n};\, t_n,\, h)
$$
donde la función $\Phi$ cumple una condición de Lipschitz adecuada.

## MML basados en cuadraturas

Usando las fórmulas clásicas habituales de integración numérica, de tipo interpolatorio, podríamos diseñar innumerables nuevos métodos de resolución numérica de problemas de PVI de tipo MML, partiendo de que la ecuación diferencial de partida $y'(t)=f(t,y(t))$ se puede expresar también de forma equivalente, como 
$$y(\tau_2)-y(\tau_1)=\int_{\tau_1}^{\tau_2} f(s,y(s))\, ds, \; \forall \tau_1, \, \tau_2 \in [a,b],$$ 
entonces podremos también escribir, $\forall 1\leq q \leq k$, notando $f_j\equiv f(t_j,y_j)$,
$$y(t_{n+k}) = y(t_{n+k-q}) + \int_{t_{n+k-q}}^{t_{n+k}} f(s,y(s))\, ds \\
\approx  y(t_{n+k-q}) + h\, \sum_{j=m}^{k-r} \beta_j \, f_{n+j}
$$
que será implícito si $r=0$, y explícito en cualquier otro caso.

Dependiendo ahora de los valores de $q,\, m, \, r$ surgen algunas conocidas familias de métodos, que revisaremos a continuación.

## Métodos de tipo Adams

Son aquellos con $q=1$, y por tanto $m=0$; y se podrían expresar como sigue:

$$ y_{n+k} = y_{n+k-1} + h\, (\beta_0\, f_n + \ldots + \beta_{k-r}\, f_{n+k-r})
$$

### Métodos de tipo Adams-Bashforth (AB)

Son métodos de tipo Adams explícitos, con exactitud máxima, y valores $q=1,\, m=0,\, r=1$.

Por ejemplo, 
- con $k=1$ tendríamos de nuevo el método de Euler: $y_{n+1} = y_{n} + h\, f_n$
- y con $k=2$: $y_{n+2} = y_{n+1} + \frac{h}{2} (3 f_{n+1} - f_{n})$

In [ ]:
%%file AdamBashforth2.py
## módulo AB2
''' Método de Adams-Bashforth (con k = 2) para resolver
    el problema de valores iniciales {y}' = {F(x,{y})}, 
    donde    {y} = {y[0],y[1],...y[N-1]}.
    X,Y    = integrate(F,x0,y0,xfinal,N).
    x0,y0  = condiciones iniciales
    xfinal = valor final de x
    N      = número de subintervalos de la partición
    h      = incremento de x usado en la integración
    F      = función suplida por el usuario que devuelve el
            array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
'''

def integrate(F,x0,y0,xfinal,N):
    
    import numpy as np
    
    def AB2(F,x0,y0,y1,h):
        return (3*F(x0+h,y1)-F(x0,y0))/2    
    
    X = np.linspace(x0,xfinal,N+1)
    h = (xfinal-x0)/N
    y1 = y0 + h*F(x0,y0); Y = [y0,y1]
    for n in range(N-1):
        Y.append(Y[n+1] + h*AB2(F,X[n],Y[n],Y[n+1],h))
    return np.array(X),np.array(Y)

In [ ]:
import AdamBashforth2

In [ ]:
# Si posteriormente tuviéramos que volver a importarlo, con Pyhton 3
# import importlib    # necesitaríamosimportar previamente este módulo
# importlib.reload(AdamBashforth2) #y ya podríamos cargarlo de nuevo cuando queramos
# importlib.reload(eulerimplicito) # 

In [ ]:
# Apliquemos pues ahora este algoritmo del método de Euler
N=10;  # con un valor concreto de n
xx, yAB2 = AdamBashforth2.integrate(f,a,ya,b,N)
# y asignémosle los arrays que devuelve a las abcisas y ordenadas

In [ ]:
# que justo a continuación podremos dibujar y comparar con la sol. exacta
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(xx,yAB2, 'bo',label = "aprox. mét. AB2")
ax.plot(xx, y_x(xx), label="sol. exacta")
ax.set_title(r"Sol. exacta:  $y(t)=%s$" % sp.latex(sol_exacta), fontsize=18)
ax.set_xlabel(r"$t$", fontsize=18)
ax.set_ylabel(r"$y$", fontsize=18)
ax.legend()
fig.tight_layout()
graf_AB2 = fig

**Ejercicios:**
1. A partir de la implementación del algoritmo del método de Adams-Bashforth anterior, realice las modificaciones oportunas para intentar obtener también las implementaciones correspondientes a otros muchos métodos de tipo Adams, como Adams-Moulton (AM), Milne-Simpson, Nyström y/o Newton-Cotes.

## Métodos Predictor-Corrector

### Método de Adams-Bashforth-Moulton de orden 5

Implementaremos a continuación un método MML Predictor-Corrector particular, combinando un predictor AB de 5 pasos con un corrector AM de 4, y aplicadon una sólo corrección en cada iteración:

P: $y_{n+5}^{(0)} = y_{n+4} + \frac{h}{720} (1901 f_{n+4} -2774 f_{n+3} +2616 f_{n+2} - 1274 f_{n+1} + 251 f_n)$

C$^1$: $y_{n+5} = y_{n+4} + \frac{h}{720} (251 f(t_{n+5},y_{n+5}^{(0)}) + 646 f_{n+4} -264 f_{n+3} +106 f_{n+2} - 19 f_{n+1}$ 

**Ejercicios:**
1. Intente la implementación del algoritmo de tipo Predictor-Corrector anterior, o cualquier otro adecuado, eligiendo convenientemente los otros métodos necesarios para proporcionar los valores previos necesarios.

## Integración Numerica de EDOs y sistemas de Ecs. Diferenciales usando SciPy  (Opcional)

In [ ]:
from scipy import integrate

In [ ]:
%matplotlib inline
# %config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
# import matplotlib as mpl
# mpl.rcParams['text.usetex'] = True

In [ ]:
import numpy as np
import sympy as sp
sp.init_printing()

In [ ]:
x = sp.symbols("x")

In [ ]:
y = sp.Function("y")

In [ ]:
f = y(x)**2 + x

In [ ]:
f_np = sp.lambdify((y(x), x), f, 'math')

In [ ]:
y0 = 0

In [ ]:
xp = np.linspace(0, 1.9, 100)

In [ ]:
xp.shape

In [ ]:
yp = integrate.odeint(f_np, y0, xp)

In [ ]:
xm = np.linspace(0, -5, 100)

In [ ]:
ym = integrate.odeint(f_np, y0, xm)

In [ ]:
def plot_direction_field(x, y_x, f_xy, x_lim=(-5, 5), y_lim=(-5, 5), ax=None):
    
    f_np = sp.lambdify((x, y_x), f_xy, 'numpy')
    
    x_vec = np.linspace(x_lim[0], x_lim[1], 20)
    y_vec = np.linspace(y_lim[0], y_lim[1], 20)
    
    if ax is None:
        _, ax = plt.subplots(figsize=(4, 4))

    dx = x_vec[1] - x_vec[0]
    dy = y_vec[1] - y_vec[0]

    for m, xx in enumerate(x_vec):
        for n, yy in enumerate(y_vec):
            Dy = f_np(xx, yy) * dx
            Dx = 0.8 * dx**2 / np.sqrt(dx**2 + Dy**2)
            Dy = 0.8 * Dy*dy / np.sqrt(dx**2 + Dy**2)
            ax.plot([xx - Dx/2, xx + Dx/2],
                    [yy - Dy/2, yy + Dy/2], 'b', lw=0.5)
    ax.axis('tight')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
plot_direction_field(x, y(x), f, ax=ax)
ax.plot(xm, ym, 'b', lw=2)
ax.plot(xp, yp, 'r', lw=2);
# fig.savefig('odeint-ejemplo-ec-escalar.pdf')

## Ecuaciones de Lotka-Volterra para poblaciones de tipo presa/predador

$$
x'(t) = a x - b x y
$$

$$
y'(t) = c x y - d y
$$

In [ ]:
a, b, c, d = 0.4, 0.002, 0.001, 0.7

In [ ]:
def f(xy, t):
    x, y = xy
    return [a * x - b * x * y,
            c * x * y - d * y]

In [ ]:
xy0 = [600, 400]

In [ ]:
t = np.linspace(0, 50, 250)

In [ ]:
xy_t = integrate.odeint(f, xy0, t)

In [ ]:
xy_t.shape

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 4))

axes[0].plot(t, xy_t[:,0], 'r', label="Presa")
axes[0].plot(t, xy_t[:,1], 'b', label="Predador")
axes[0].set_xlabel("Tiempo")
axes[0].set_ylabel("Numero de animales")
axes[0].legend()

axes[1].plot(xy_t[:,0], xy_t[:,1], 'k')
axes[1].set_xlabel("Numero de presas")
axes[1].set_ylabel("Numero de predadores")
fig.tight_layout()
fig.savefig('cap9-lokta-volterra.pdf')

## Ecuaciones de Lorenz

$$
x'(t) = \sigma(y - x)
$$
$$
y'(t) = x(\rho - z) - y 
$$
$$
z'(t) = x y - \beta z
$$

In [ ]:
def f(xyz, t, rho, sigma, beta):
    x, y, z = xyz
    return [sigma * (y - x),
            x * (rho - z) - y,
            x * y - beta * z]

In [ ]:
rho = 28
sigma = 8
beta = 8/3.0

In [ ]:
t = np.linspace(0, 25, 10000)

In [ ]:
xyz0 = [1.0, 1.0, 1.0]

In [ ]:
xyz1 = integrate.odeint(f, xyz0, t, args=(rho, sigma, beta))

In [ ]:
xyz2 = integrate.odeint(f, xyz0, t, args=(rho, sigma, 0.6*beta))

In [ ]:
xyz3 = integrate.odeint(f, xyz0, t, args=(rho, 2*sigma, 0.6*beta))

In [ ]:
xyz3.shape

In [ ]:
from mpl_toolkits.mplot3d.axes3d import Axes3D

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 3.5), subplot_kw={'projection': '3d'})

for ax, xyz, c in [(ax1, xyz1, 'r'), (ax2, xyz2, 'b'), (ax3, xyz3, 'g')]:
    ax.plot(xyz[:,0], xyz[:,1], xyz[:,2], c, alpha=0.5)
    ax.set_xlabel('$x$', fontsize=16)
    ax.set_ylabel('$y$', fontsize=16)
    ax.set_zlabel('$z$', fontsize=16)
    ax.set_xticks([-15, 0, 15])
    ax.set_yticks([-20, 0, 20])
    ax.set_zticks([0, 20, 40])

fig.tight_layout()
fig.savefig('cap9-ecuaciones_Lorentz.pdf')

## Muelles acoplados y amortigüados

Como ecuaciones de segundo orden:

\begin{eqnarray}
m_1 x_1''(t) + \gamma_1 x_1'(t) + k_1 (x_1(t) - l_1) - k_2 (x_2(t) - x_1(t) - l_2) &=& 0\\
m_2 x_2''(t) + \gamma_2 x_2' + k_2 (x_2 - x_1 - l_2) &=& 0
\end{eqnarray}

O de forma estándard:

\begin{align}
y_1'(t) &= y_2(t) \\
y_2'(t) &= -\gamma_1/m_1 y_2(t) - k_1/m_1 (y_1(t) - l_1) + k_2 (y_3(t) - y_1(t) - l_2)/m_1 \\
y_3'(t) &= y_4(t) \\
y_4'(t) &= - \gamma_2 y_4(t)/m_2 - k_2 (y_3(t) - y_1(t) - l_2)/m_2 \\
\end{align}

In [ ]:
def f(t, y, args):
    m1, k1, g1, m2, k2, g2 = args
    
    return [y[1], 
            - k1/m1 * y[0] + k2/m1 * (y[2] - y[0]) - g1/m1 * y[1], 
            y[3], 
            - k2/m2 * (y[2] - y[0]) - g2/m2 * y[3] ]

In [ ]:
m1, k1, g1 = 1.0, 10.0, 0.5

In [ ]:
m2, k2, g2 = 2.0, 40.0, 0.25

In [ ]:
args = (m1, k1, g1, m2, k2, g2)

In [ ]:
y0 = [1.0, 0, 0.5, 0]

In [ ]:
t = np.linspace(0, 20, 1000)

In [ ]:
r = integrate.ode(f)

In [ ]:
r.set_integrator('lsoda');

In [ ]:
r.set_initial_value(y0, t[0]);

In [ ]:
r.set_f_params(args);

In [ ]:
dt = t[1] - t[0]
y = np.zeros((len(t), len(y0)))
idx = 0
while r.successful() and r.t < t[-1]:
    y[idx, :] = r.y
    r.integrate(r.t + dt)
    idx += 1

In [ ]:
fig = plt.figure(figsize=(10, 4))
ax1 = plt.subplot2grid((2, 5), (0, 0), colspan=3)
ax2 = plt.subplot2grid((2, 5), (1, 0), colspan=3)
ax3 = plt.subplot2grid((2, 5), (0, 3), colspan=2, rowspan=2)

ax1.plot(t, y[:, 0], 'r')
ax1.set_ylabel('$x_1$', fontsize=18)
ax1.set_yticks([-1, -.5, 0, .5, 1])

ax2.plot(t, y[:, 2], 'b')
ax2.set_xlabel('$t$', fontsize=18)
ax2.set_ylabel('$x_2$', fontsize=18)
ax2.set_yticks([-1, -.5, 0, .5, 1])

ax3.plot(y[:, 0], y[:, 2], 'k')
ax3.set_xlabel('$x_1$', fontsize=18)
ax3.set_ylabel('$x_2$', fontsize=18)
ax3.set_xticks([-1, -.5, 0, .5, 1])
ax3.set_yticks([-1, -.5, 0, .5, 1])

fig.tight_layout()
# fig.savefig('cap9-muelles-acoplados.pdf')

#### Realicemos ahora el mismo cálculo anterior, pero especificando también el Jacobiano:

In [ ]:
def jac(t, y, args):
    m1, k1, g1, m2, k2, g2 = args
    
    return [[0, 1, 0, 0], 
            [- k1/m1 - k2/m1, - g1/m1 * y[1], k2/m1, 0],
            [0, 0, 1, 0],
            [k2/m2, 0, - k2/m2, - g2/m2]]

In [ ]:
r = integrate.ode(f, jac).set_f_params(args).set_jac_params(args).set_initial_value(y0, t[0])

In [ ]:
dt = t[1] - t[0]
y = np.zeros((len(t), len(y0)))
idx = 0
while r.successful() and r.t < t[-1]:
    y[idx, :] = r.y
    r.integrate(r.t + dt)
    idx += 1

In [ ]:
fig = plt.figure(figsize=(10, 4))
ax1 = plt.subplot2grid((2, 5), (0, 0), colspan=3)
ax2 = plt.subplot2grid((2, 5), (1, 0), colspan=3)
ax3 = plt.subplot2grid((2, 5), (0, 3), colspan=2, rowspan=2)

ax1.plot(t, y[:, 0], 'r')
ax1.set_ylabel('$x_1$', fontsize=18)
ax1.set_yticks([-1, -.5, 0, .5, 1])

ax2.plot(t, y[:, 2], 'b')
ax2.set_xlabel('$t$', fontsize=18)
ax2.set_ylabel('$x_2$', fontsize=18)
ax2.set_yticks([-1, -.5, 0, .5, 1])

ax3.plot(y[:, 0], y[:, 2], 'k')
ax3.set_xlabel('$x_1$', fontsize=18)
ax3.set_ylabel('$x_2$', fontsize=18)
ax3.set_xticks([-1, -.5, 0, .5, 1])
ax3.set_yticks([-1, -.5, 0, .5, 1])

fig.tight_layout()

### Mismos cálculos, pero usando previamente SymPy para definir el problema para SciPy

In [ ]:
L1 = L2 = 0
t = sp.symbols("t")
m1, k1, b1 = sp.symbols("m_1, k_1, b_1")
m2, k2, b2 = sp.symbols("m_2, k_2, b_2")

In [ ]:
x1 = sp.Function("x_1")
x2 = sp.Function("x_2")

In [ ]:
ode1 = sp.Eq(m1 * x1(t).diff(t,t,) + b1 * x1(t).diff(t) + k1*(x1(t)-L1) - k2*(x2(t)-x1(t) - L2),0)

In [ ]:
ode2 = sp.Eq(m2 * x2(t).diff(t,t,) + b2 * x2(t).diff(t) + k2*(x2(t)-x1(t)-L2),0)

In [ ]:
params = {m1: 1.0, k1: 10.0, b1: 0.5,
          m2: 2.0, k2: 40.0, b2: 0.25}

In [ ]:
args

In [ ]:
y1 = sp.Function("y_1")
y2 = sp.Function("y_2")
y3 = sp.Function("y_3")
y4 = sp.Function("y_4")

In [ ]:
varchange = {x1(t).diff(t, t): y2(t).diff(t), 
             x1(t): y1(t),
             x2(t).diff(t, t): y4(t).diff(t), 
             x2(t): y3(t)}

In [ ]:
(ode1.subs(varchange).lhs, ode2.subs(varchange).lhs)

In [ ]:
ode3 = y1(t).diff(t) - y2(t)

In [ ]:
ode4 = y3(t).diff(t) - y4(t)

In [ ]:
vcsol = sp.solve((ode1.subs(varchange), ode2.subs(varchange), ode3, ode4),
                    (y1(t).diff(t), y2(t).diff(t), y3(t).diff(t), y4(t).diff(t)))

In [ ]:
vcsol

In [ ]:
ode_rhs = sp.Matrix([y1(t).diff(t), y2(t).diff(t), y3(t).diff(t), y4(t).diff(t)]).subs(vcsol)

In [ ]:
y = sp.Matrix([y1(t), y2(t), y3(t), y4(t)])

In [ ]:
sp.Eq(y.diff(t), ode_rhs)

In [ ]:
f_np = sp.lambdify((t, y), ode_rhs.subs(params), 'numpy')

In [ ]:
y0 = [1.0, 0, 0.5, 0]
t = np.linspace(0, 20, 1000)

r = integrate.ode(f_np)
r.set_integrator('lsoda');
r.set_initial_value(y0, t[0]);

dt = t[1] - t[0]
y = np.zeros((len(t), len(y0)))
idx = 0
while r.successful() and r.t < t[-1]:
    y[idx, :] = r.y
    r.integrate(r.t + dt)
    idx += 1

In [ ]:
fig = plt.figure(figsize=(10, 4))
ax1 = plt.subplot2grid((2, 5), (0, 0), colspan=3)
ax2 = plt.subplot2grid((2, 5), (1, 0), colspan=3)
ax3 = plt.subplot2grid((2, 5), (0, 3), colspan=2, rowspan=2)

ax1.plot(t, y[:, 0], 'r')
ax1.set_ylabel('$x_1$', fontsize=18)
ax1.set_yticks([-1, -.5, 0, .5, 1])

ax2.plot(t, y[:, 2], 'b')
ax2.set_xlabel('$t$', fontsize=18)
ax2.set_ylabel('$x_2$', fontsize=18)
ax2.set_yticks([-1, -.5, 0, .5, 1])

ax3.plot(y[:, 0], y[:, 2], 'k')
ax3.set_xlabel('$x_1$', fontsize=18)
ax3.set_ylabel('$x_2$', fontsize=18)
ax3.set_xticks([-1, -.5, 0, .5, 1])
ax3.set_yticks([-1, -.5, 0, .5, 1])

fig.tight_layout()

## Doble péndulo

http://scienceworld.wolfram.com/physics/DoublePendulum.html

$$
(m_1+m_2) l_1\theta_1'' + m_2l_2\theta_2''\cos(\theta_1-\theta_2)
+ m_2l_2(\theta_2')^2\sin(\theta_1-\theta_2)+g(m_1+m_2)\sin(\theta_1) = 0
$$

$$
m_2l_2\theta_2'' + m_2l_1\theta_1''\cos(\theta_1-\theta_2) - m_2l_1 (\theta_1')^2 \sin(\theta_1-\theta_2)
+m_2g\sin(\theta_2) = 0
$$

In [ ]:
t, g, m1, l1, m2, l2 = sp.symbols("t, g, m_1, l_1, m_2, l_2")

In [ ]:
theta1, theta2 = sp.symbols("theta_1, theta_2", cls=sp.Function)

In [ ]:
ode1 = sp.Eq((m1+m2)*l1 * theta1(t).diff(t,t) +
                m2*l2 * theta2(t).diff(t,t) +
                m2*l2 * theta2(t).diff(t)**2 * sp.sin(theta1(t)-theta2(t)) + 
                g*(m1+m2) * sp.sin(theta1(t)),0)
ode1

In [ ]:
ode2 = sp.Eq(m2*l2 * theta2(t).diff(t,t) +
                m2*l1 * theta1(t).diff(t,t) * sp.cos(theta1(t)-theta2(t)) -
                m2*l1 * theta1(t).diff(t)**2 * sp.sin(theta1(t) - theta2(t)) +
                m2*g * sp.sin(theta2(t)),0)
ode2

In [ ]:
# Esto resultará infructuoso, ya que Sympy no puede resolver simbólicamente estas EDOs
try:
    sp.dsolve(ode1, ode2)
except Exception as e:
    print(e)

In [ ]:
y1, y2, y3, y4 = sp.symbols("y_1, y_2, y_3, y_4", cls=sp.Function)

In [ ]:
varchange = {theta1(t).diff(t, t): y2(t).diff(t), 
             theta1(t): y1(t),
             theta2(t).diff(t, t): y4(t).diff(t), 
             theta2(t): y3(t)}

In [ ]:
ode1_vc = ode1.subs(varchange)

In [ ]:
ode2_vc = ode2.subs(varchange)

In [ ]:
ode3 = y1(t).diff(t) - y2(t)

In [ ]:
ode4 = y3(t).diff(t) - y4(t)

In [ ]:
y = sp.Matrix([y1(t), y2(t), y3(t), y4(t)])

In [ ]:
vcsol = sp.solve((ode1_vc, ode2_vc, ode3, ode4), y.diff(t), dict=True)

In [ ]:
f = y.diff(t).subs(vcsol[0])

In [ ]:
sp.Eq(y.diff(t), f)

In [ ]:
params = {m1: 5.0, l1: 2.0,
          m2: 1.0, l2: 1.0, g: 10.0}

In [ ]:
f_np = sp.lambdify((t, y), f.subs(params), 'numpy')

In [ ]:
jac = sp.Matrix([[fj.diff(yi) for yi in y] for fj in f])

In [ ]:
jac_np = sp.lambdify((t, y), jac.subs(params), 'numpy')

In [ ]:
y0 = [2.0, 0, 0.0, 0]

In [ ]:
t = np.linspace(0, 20, 1000)

In [ ]:
jac_np(0, y0)

In [ ]:
r = integrate.ode(f_np, jac_np).set_initial_value(y0, t[0]);
 
dt = t[1] - t[0]
y = np.zeros((len(t), len(y0)))
idx = 0
while r.successful() and r.t < t[-1]:
    y[idx, :] = r.y
    r.integrate(r.t + dt)
    idx += 1

In [ ]:
fig = plt.figure(figsize=(10, 4))
ax1 = plt.subplot2grid((2, 5), (0, 0), colspan=3)
ax2 = plt.subplot2grid((2, 5), (1, 0), colspan=3)
ax3 = plt.subplot2grid((2, 5), (0, 3), colspan=2, rowspan=2)

ax1.plot(t, y[:, 0], 'r')
ax1.set_ylabel(r'$\theta_1$', fontsize=18)

ax2.plot(t, y[:, 2], 'b')
ax2.set_xlabel('$t$', fontsize=18)
ax2.set_ylabel(r'$\theta_2$', fontsize=18)

ax3.plot(y[:, 0], y[:, 2], 'k')
ax3.set_xlabel(r'$\theta_1$', fontsize=18)
ax3.set_ylabel(r'$\theta_2$', fontsize=18)

fig.tight_layout()

In [ ]:
theta1_np, theta2_np = y[:, 0], y[:, 2]

In [ ]:
x1 = params[l1] * np.sin(theta1_np)
y1 = -params[l1] * np.cos(theta1_np)
x2 = x1 + params[l2] * np.sin(theta2_np)
y2 = y1 - params[l2] * np.cos(theta2_np)

In [ ]:
fig = plt.figure(figsize=(10, 4))
ax1 = plt.subplot2grid((2, 5), (0, 0), colspan=3)
ax2 = plt.subplot2grid((2, 5), (1, 0), colspan=3)
ax3 = plt.subplot2grid((2, 5), (0, 3), colspan=2, rowspan=2)

ax1.plot(t, x1, 'r')
ax1.plot(t, y1, 'b')
ax1.set_ylabel('$x_1, y_1$', fontsize=18)
ax1.set_yticks([-3, 0, 3])

ax2.plot(t, x2, 'r')
ax2.plot(t, y2, 'b')
ax2.set_xlabel('$t$', fontsize=18)
ax2.set_ylabel('$x_2, y_2$', fontsize=18)
ax2.set_yticks([-3, 0, 3])

ax3.plot(x1, y1, 'r')
ax3.plot(x2, y2, 'b', lw=0.5)
ax3.set_xlabel('$x$', fontsize=18)
ax3.set_ylabel('$y$', fontsize=18)
ax3.set_xticks([-3, 0, 3])
ax3.set_yticks([-3, 0, 3])

fig.tight_layout()
# fig.savefig('ch9-double-pendulum.pdf')

## Versiones

In [ ]:
%reload_ext version_information

In [ ]:
%version_information numpy, scipy, sympy, matplotlib